# General Introduction

This section provides you with a general introduction to these hands-on sessions.

The idea is that you will work on your own, following the instructions reported in the notebooks. The instructors will be available to answer questions, discuss issues and problems individually and/or with the rest of the class. At the beginning of each notebook, there's a small summary of its content. You should be able to complete the whole notebook in 2 hours.

Troughout the notebooks, whenever you find a blue box like this one:

<div class="alert alert-block alert-info">
<b>Question:</b> What should you do when you find a blue box?
</div>

It means that there's a question for you to think about. Instead, a yellow box like this one:

<div class="alert alert-block alert-warning">
<b>Task:</b> Complete the following cell by adding X lines of code.
</div>

means that there's some code for you to write.


# Hands-on #1: Training and DNN Optimization on Multi-Patient Dataset

In this notebook, you will:
1. Load a pre-collected and pre-processed dataset of sEMG signals for gesture recognition containing data from multiple patients.
2. Import a pre-defined CNN from the state-of-the-art for this task
3. Train it on the data and evaluate its performance.
4. Apply a Channel Pruning algorithm (PIT) on the model to trade-off accuracy and size.


# Part 1: First DNN Training

## Initial Setup

Let's start by importing the required libraries:

In [1]:
import os
import sys
import random
import copy
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torcheval.metrics import MulticlassAccuracy, Mean
from torchinfo import summary

from utils.data import EMGDataset
from utils.model import TEMPONet
from utils.train import CheckPoint, EarlyStopping, try_load_checkpoint
from utils.plot import plot_signal, plot_learning_curves, plot_conf_matrix, plot_pareto

Next, we perform some initial setup operations. Namely, we set some constants for file paths and define `TRAINING_CONFIG`, a dictionary containing basic training configurations, which we'll use later. Then, we call a function defined in `utils/train.py` which applies random seeds to maximize reproducibility. Lastly, we set the training device (GPU if available, CPU otherwise).

In [2]:
DATASET_DIR = Path("./dataset")
SAVE_DIR = Path(f"experiments/hands_on_1/")

TRAINING_CONFIG = {
    'epochs': 150,
    'batch_size': 32,
    'learning_rate': 0.0005,
    'nas_learning_rate': 0.001,
    'label_smoothing': 0.25,
    'patience': 30
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Working on: {device}")

## The Dataset

Our pre-collected dataset contains surface Electromiography (EMG) signals collected from 8 electrodes positioned on the user's forearm. A picture of the data collection setup (that you will have to replicate in Hands-on #2) is shown in the picture below.

<br>
<center><img src="./assets/setup.png" alt="setup" class="bg-primary" width="200px"></center>
<center> Fig.1: Data collection setup. </center>
<br>

The dataset has the following characteristics:
* It contains 3 collection sessions for each subject, acquired at different times during the day.
* In each session, subjects were asked to perform 8 different gestures, each repeated 5 times.
* Every repetition lasted 8s, separated by 5s of rest, with the arm and hand in a neutral position.
* Data are sampled at 500 Hz, from all the 8 electrodes.

The goal of the training will be to recognize each of the 8 gestures, plus the rest position. So, we will frame the problem as a **9-class classification**. The following figure displays the eight gesture classes:
<br>
<center><img src="./assets/classes.png" align="center" alt="gesture classes" class="bg-primary" width="80%"></center>
<center> Fig.2: The eight gestures (in order of class ID, left to right, top to bottom). </center>
<br>


### Pre-processing

We feed our DNNs with EMG data after an initial pre-processing phase. In particular, we apply the following pre-processing steps:
1. We apply a 4-th order high-pass Butterworth **filter** to eliminate the continuous component from the signal.
2. We **trim** the initial and final part of each session (before the first gesture and after the last one), which contain transients.
3. We apply a **min-max** normalization to have the same value ranges for all channels.
4. We create windows of 600ms length (300 samples), with a 70% overlap between consecutive windows, to balance data quantity and diversity. We only select "steady windows", that is, those associated to a single class label, excluding transients. 

All three normalization steps have already been applied to the pre-collected dataset. Later, we will see how you can repeat these steps on your own data.

### Splitting

The default way to train state-of-the-art sEMG-based gesture recognition systems involves *personalized, per-subject training*. In other words, the trained DNN is expected to learn from data of Subject X, and then generalize well on data from the same subject. In this first session, however, we will train our networks on **data from multiple patients** simultaneously. This is because the goal here is *finding a single, optimized DNN architecture* for this task, which is expected to work well for multiple patients.

However, it is important to make sure that our model generalizes well at least over *different data collection sessions*. This ensures us that if the EMG electodes are removed from the arm and later re-positioned on it in a different day (following the same setup) the DNN will still output meaningful predictions. Accordingly, we will split our data as follows. For each patient:
* All 5 gestures repetitions from session_1 will go in the training set.
* The first 3 repetitions from session_2 will also go in the training set, while the last 2 repetitions will go in the validation set.
* All 5 repetitions from session_3 will be used as test set.

In Hands-on #3, after having collected your own data, we will use a similar setup to train a personalized model, which will hopefully perform better.

### Visualize the Data

Let's load the data:

In [3]:
train_data = np.load(DATASET_DIR / "train_data.npy")
train_labels = np.load(DATASET_DIR / "train_labels.npy")
val_data = np.load(DATASET_DIR / "val_data.npy")
val_labels = np.load(DATASET_DIR / "val_labels.npy")
test_data = np.load(DATASET_DIR / "test_data.npy")
test_labels = np.load(DATASET_DIR / "test_labels.npy")
print(f"Training data shape: {train_data.shape}, labels: {train_labels.shape}")
print(f"Validation data shape: {val_data.shape}, labels: {val_labels.shape}")
print(f"Test data shape: {test_data.shape}, labels: {test_labels.shape}")

The next function plots two signal windows side by side. It is defined in `./utils/plot.py` if you're interested in the code (but it's nothing special). Let's use it to plot an example of rest window and one of the gestures. You can of course also try other windows:

In [4]:
EXAMPLE_REST = 1000
EXAMPLE_GESTURE = 2500
plot_signal(EXAMPLE_REST, EXAMPLE_GESTURE, train_data, train_labels)

## Prepare Data for Training

We will wrap our data in a simple PyTorch `Dataset` class. You can find its definition in the file: `utils/data.py`.
It essentially loads the data and prepares it for usage with our DNNs. Namely, there are two key preparation operations:
- First, the data is reorganized from a $(N, T, C)$ layout to $(N, C, T)$, where $N$ is the window index, $T$ the window length and $C$ the number of channels.
- Moreover, an additional dummy dimension is added, obtainin a final layout of: $(N, T, C, 1)$.

The first axis permutation is needed since torch expects features/channels to be the outermost dimension for each sample. The additional axis, instead, allows us to use a 2D CNN for our trainings. We essentially treat each EMG window as a 8-channel "image" with a single row and $T=300$ columns. The reason for this is that 1D CNN layers are not fully supported by our AI Compiler, which we'll use in a following hands-on session to generate C code for GAP9 starting from the networks that you will train now.

Let's create three instances of this dataset class for our training, validation and test sets.
<div class="alert alert-block alert-warning">
<b>Task:</b> Complete the following function for the validation and test datasets. Verify that the length of the two datasets did not change.
</div>

In [5]:
train_ds = EMGDataset(train_data, train_labels)
# create the datasets for validation and test sets. call them val_ds and test_ds respectively (expected: 2 lines)
# YOUR_CODE_START


# YOUR_CODE_END

# save the input shape for later use
input_shape = train_ds[0][0].shape

# print the length of each dataset (using the len() builtin). Verify that the length did not change (expected: 3 lines)
# YOUR_CODE_START



# YOUR_CODE_END

Next, let's generate torch `DataLoader` classes, setthing the batch size and some other parameters for reproducibility and speed. Don't worry about the details if you're not familiar with them. The next function takes as first parameter the training configuration dictionary defined at the beginning of the notebook.


In [6]:
def build_dataloaders(config, train_ds, val_ds, test_ds, num_workers=4):
    generator = torch.Generator().manual_seed(42)

    train_loader = DataLoader(
        train_ds, batch_size=config['batch_size'], shuffle=True, pin_memory=True, num_workers=num_workers,
        generator=generator, drop_last=True
    )

        
    val_loader = DataLoader(
        val_ds, batch_size=config['batch_size'], shuffle=False, pin_memory=True, num_workers=num_workers,
        generator=generator,
    )
    
    test_loader = DataLoader(
        test_ds, batch_size=config['batch_size'], shuffle=False, num_workers=num_workers,
        generator=generator, 
    )

    return train_loader, val_loader, test_loader

We then the function and check the result.

In [7]:
train_dl, val_dl, test_dl = build_dataloaders(TRAINING_CONFIG, train_ds, val_ds, test_ds, num_workers=4)
print(f"Training data-loader length: {len(train_dl)}")
print(f"Validation data-loader length: {len(val_dl)}")
print(f"Test data-loader length: {len(test_dl)}")

## The "Seed" DNN

We're now ready to train a DNN on our data. In this first experiment, we will use a network from the state-of-the-art called TEMPONet, introduced in [this](https://ieeexplore.ieee.org/document/8930945) paper. Below is a diagram describing the network architecture, taken from the original paper.

<br>
<center><img src="./assets/temponet.png" alt="setup" class="bg-primary" width="50%"></center>
<center> Fig.3: TEMPONet architecture. </center>
<br>

Note that the network used in this notebook differs slightly from the original TEMPONet. For instance, our input has a lower number of channels (8 vs 14). We also replace Average Pooling with Max Pooling (more hardware-friendly). Furthermore, as already, we turned it into a 2D CNN (although fully equivalent to the original 1D model) for practical reasons.

The network is defined in the file `utils/train.py`. Take a look at it and verify you understand it fully. In the next cell, we create an instance of TEMPONet and move it to our training device. We wrap the code in a function:



In [8]:
def get_model():
    model = TEMPONet()
    model = model.to(device)
    return model

Next, we call the function, and print a summary of the model using the `torchinfo` library:


In [9]:
model = get_model()
single_input_batch = (1,) + input_shape
print(summary(model, single_input_batch))


<div class="alert alert-block alert-info">
<b>Questions:</b> 1) How is the network structured? Look in particular at the number of channels. 2) How much memory would this model take? Is it compatible with the GAP9 platform on which we want to deploy?
</div>

## Training Functions

### Loss Function

As our loss function, we use a standard Categorical Cross-Entropy for multi-class classification. But with two important caveats.

First, in our dataset, one class (the "rest" position) is significantly over-represented with respect to the other 8. Therefore, we want to assign **class-dependent weights** to our samples, inversely proportional to the class frequency in the training set. In other words, we want to assign higher importance (i.e. a larger weight) to the 8 gestures, and lower importance to the rest class. This avoids that our DNN only learns to accurately recognize "rest", which would not be practically useful.

Second, we apply **label smoothing regularization**, a technique which modifies the training target to be a weighted mix of the ground truth class, and a uniform distribution over all classes. This technique, first described in [this](https://arxiv.org/pdf/1512.00567) paper, avoids over-confidence in the model predictions and improves regularization.

We wrap the loss construction in a function so that we can recall it later.


In [10]:
def get_criterion(train_dl, config, device):
    # compute the number of windows belonging to each class
    classes, class_frequencies = np.unique(train_dl.dataset.Y, return_counts=True)
    
    # derive corresponding loss weights as w_class_i = tot_windows / class_i_windows
    loss_weights = sum(class_frequencies) / (class_frequencies)
    
    # create an instance of the torch.nn.CrossEntropyLoss with weight and label_smoothing and return it 
    crit = torch.nn.CrossEntropyLoss(weight=torch.Tensor(loss_weights).to(device), label_smoothing=config['label_smoothing'])
    return crit

### Optimizer

As the optimizer for training, we use the classic Adam algorithm. Again, we wrap its construction in a function:

In [11]:
def get_optimizer(model, config):
    # return an Adam optimizer set to optimize model.parameters() with the correct learning rate
    return torch.optim.Adam(model.parameters(), lr=config['learning_rate'])


We are now ready to train. For this, let's define two functions, respectively to train the DNN for one epoch and to evaluate it on unseen data.

### Train One Epoch

This function takes as input the `model`, the loss function (`criterion`), the `optimizer`, a dataloader (`train_dl`), and the training `device`. 
We also need to know the number of classes in our dataset (`num_classes`) to compute one of the metrics. It then iterates over the dataloader applying the standard sequence of training operations (forward pass, loss computation, backward pass and weight updates), periodically logging information on a file (set to stdout by default). Lastly, it returns the value of the loss and metrics of interest. Some things to note here:
- We use `torcheval`'s classes to keep track of the loss and metrics minibatch by minibatch. Look at the library documentation for details on each function, if interested.
- Importantly, besides the loss and the standard (micro-average) accuracy, we also measure the **macro-average** accuracy.

Macro-average accuracy is defined as the unweighted average of the accuracy obtained on each of the 9 classes separately. This is a more relevant metric in case of unbalanced datasets (like ours) because the standard accuracy would be biased by the most represented classes. Accordingly, *we will use this as our main metric of interest hereinafter*. 

In [12]:
def train_one_epoch(model, criterion, optimizer, train_dl, device, num_classes, file=sys.stdout):
    # initialize loss and metrics trackers
    avg_loss = Mean(device=device)
    avg_acc = MulticlassAccuracy(device=device)
    avg_macro_acc = MulticlassAccuracy(device=device, average='macro', num_classes=num_classes)

    # set the model in training mode
    model.train()
    step = 0
    # iterate over the dataset
    for sample, target in train_dl:
        # move data and labels to the training device
        sample, target = sample.to(device), target.to(device)
        # training iteration: forward, loss, backward, weight update
        output = model(sample)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # update loss and metrics
        avg_loss.update(loss)
        avg_acc.update(output, target)
        avg_macro_acc.update(output, target)
        
        if step % 10 == 0:
            print(f"Training... Loss: {avg_loss.compute():.2f}, Acc: {avg_acc.compute():.2f}, Macro Acc: {avg_macro_acc.compute():.2f}", end='\r', file=file)
        step += 1
        
    return avg_loss.compute().cpu().numpy(), avg_acc.compute().cpu().numpy(), avg_macro_acc.compute().cpu().numpy()

### Evaluate

The evaluation function is essentially identical to the previous one, but it is run in eval mode and without buffering information required for back-propagation. Furthermore, it does not do any logging:

In [13]:
def evaluate(model, criterion, eval_dl, device, num_classes):
    avg_loss = Mean(device=device)
    avg_acc = MulticlassAccuracy(device=device)
    avg_macro_acc = MulticlassAccuracy(device=device, average='macro', num_classes=num_classes)

    model.eval()
    with torch.no_grad():
        for sample, target in eval_dl:
            sample, target = sample.to(device), target.to(device)
            output = model(sample)
            loss = criterion(output, target)
            avg_loss.update(loss)
            avg_acc.update(output, target)
            avg_macro_acc.update(output, target)
    return avg_loss.compute().cpu().numpy(), avg_acc.compute().cpu().numpy(), avg_macro_acc.compute().cpu().numpy()

## Main Training Loop

The next cell contains the main training loop. Once again, we wrap everything in a function for later retrieval. This function essentially loops for a certain number of *epochs*, repeatedly calling the `train_one_epoch` and `evaluate` functions on the training and validation sets respectively. We also log the complete metrics at the end of each epoch, both in text form and in a pandas dataframe.

The code also deals with **checkpointing** and **early stopping**, using two custom classes defined in `./utils/train.py`. You can have a look at them if you want, to understand how they work. The minimum required information is provided in the collapsible box below.

As a last note, the initial "if" statement uses a custom function to look for a checkpoint called `best.ckp`. If it finds it, it directly loads the model and returns it, skipping the training entirely. This is added for convenience in case you want to run the notebook multiple times, without having to wait for all trainings to complete. Note that no history would be returned in this case.

<details>
    
  <summary>Details on CheckPoint and EarlyStopping</summary>

-----
The `CheckPoint` class allows you to save the weights and training information of your DNN periodically, and do it (optionally) only if the model's performance improved!! We will save this info after every epoch.  The class also allows you to restore the *best* version of the DNN (i.e., the one that performs best on the validation set).

The constructor parameters are:
- The directory where you want to save your checkpoints
- The two pieces of information that we want to be able to save and restore, i.e., the `model` and corresponding `optimizer`
- A flag that specifies if the metric that we will later provide to the class to determine the "best" DNN version, shall be maximized (`max`) or minimized (`min`). We set this to `max` since we will use the macro-average accuracy as our target metric.
- A format string, that allows you to generate variable checkpoint names, either depending on the `epoch` or on the metric's value (`val`). For instance, `ck_{epoch:03d}.pt` means that each checkpoint will be created with a name starting with `ck_` and followed by the epoch number on 3 digits, zero-filled. For instance, `ck_001.pt` for epoch n. 1.

The second utility class is `EarlyStopping`, which allows you to easily check if the DNN has stopped improving for a certain number of epochs (the "patience"), and stop the training when that happens. This speeds up training and limits overfitting. The constructor, in this case, simply requires to define this parameter, and again, whether the target metric shall be maximized or minimized.

</details>



In [14]:
def training_loop(checkpoint_dir, config, model, criterion, train_dl, val_dl, device, num_classes=9, file=sys.stdout):

    # look for a "best.ckp" within "checkpoint_dir" and skip the training if found
    if try_load_checkpoint(model, checkpoint_dir, device):
        print(f"Skipping training and loading pre-cooked model from: {checkpoint_dir}")
        return None

    # initialize optimizer, checkpoint, earlystopping and history
    optimizer = get_optimizer(model, config)
    checkpoint = CheckPoint(checkpoint_dir, model, optimizer, 'max', fmt='ck_{epoch:03d}.ckp')
    earlystop = EarlyStopping(patience=config['patience'], mode='max')
    history = []

    # training loop
    for epoch in range(config['epochs']):
        # repeatedly call train_one_epoch and evaluate
        t_loss, t_acc, t_macro_acc = train_one_epoch(model, criterion, optimizer, train_dl, device, num_classes, file)
        v_loss, v_acc, v_macro_acc = evaluate(model, criterion, val_dl, device, num_classes)

        # logging
        history.append((epoch+1, t_loss, t_acc, t_macro_acc, v_loss, v_acc, v_macro_acc))
        print(f"Epoch: {epoch+1}, Loss: {t_loss:.2f}, Acc: {t_acc:.2f}, Macro Acc: {t_macro_acc:.2f}",
              f"Val Loss: {v_loss:.2f}, Val Acc: {v_acc:.2f}, Val Macro Acc: {v_macro_acc:.2f}", file=file)
        
        # save checkpoint
        checkpoint(epoch, v_macro_acc)
        
        # check if we need to early-stop
        if earlystop(v_macro_acc):
            print("Stopped at epoch {} because of early stopping".format(epoch + 1))
            break

    # turn history into a dataframe
    cols = ['epoch','loss','acc','macro_acc', 'val_loss', 'val_acc', 'val_macro_acc']
    history = pd.DataFrame(history, columns=cols)
    history.to_csv(checkpoint_dir / "history.csv", index=False)


    # restore the best model from the checkpoint and save it as "best.ckp" for later
    checkpoint.load_best()
    checkpoint.save(checkpoint_dir / 'best.ckp')
    
    return history

The next cell runs the training. Since this will take some time, you are suggested to keep on reading the next sections of the hands-on while you wait...

In [15]:
criterion = get_criterion(train_dl, TRAINING_CONFIG, device) # the loss function is defined externally because we need it later
history = training_loop(SAVE_DIR / 'seed', TRAINING_CONFIG, model, criterion, train_dl, val_dl, device, num_classes=9)

## Analyzing the Training Output

The next function plots the training and validation loss over the training epochs. It can be useful to debug training, e.g., to verify that both losses are decreasing as expected.

In [16]:
plot_learning_curves(history)

<div class="alert alert-block alert-info">
<b>Question:</b> How would you evaluate this training? Are we over/under-fitting? What could we do about it? (Just think about possible improvements, but please keep them for later and move on)
</div>

We can evaluate the best version of our model on the test set using our previously defined evaluate function.


In [17]:
# call the evaluate function to get the test set metrics, then print them
test_loss, test_acc, test_macro_acc = evaluate(model, criterion, test_dl, device, num_classes=9)
print(f"Test Loss: {test_loss:.2f}, Test Acc: {test_acc:.2f}, Test Macro Acc: {test_macro_acc:.2f}")

Another interesting visualization of our results is the **confusion matrix**, which shows us how the DNN is behaving on each class. Call the next function to visualize it:

In [18]:
plot_conf_matrix(model, test_dl, device)

<div class="alert alert-block alert-info">
<b>Question:</b> Look at the confusion matrix. What classes are most mistaken for one another? Are these errors reasonable, considering what each EMG electrode measures?
</div>

Let's save the seed model to disk:

In [19]:
torch.save(model, SAVE_DIR / 'final_model_seed.pt')

# Part 2: DNN Optimization

## PLiNIO Introduction

For the following parts of this notebook, we will use an open-source DNN optimization library called **PLiNIO** (Plug-and Play Lightweight Neural Inference Optimization), that we developed internally at Politecnico di Torino. You can find PLiNIO's code and documentation at [this](https://github.com/eml-eda/plinio) link.



<br>
<center><img src="./assets/plinio_logo.png" alt="PLiNIO" class="bg-primary" width="50%"></center>
<br>


Of course, there are many alternatives to this library, some much more complete and capable. Examples include Microsoft's [NNI](https://github.com/microsoft/nni) or Freiburg's [NASLib](https://github.com/automl/NASLib). We use PLiNIO for the following reasons:
- It was designed with usability in mind, and requires miminal modifications to your existing pytorch training code, which is good for time-constrained sessions like ours.
- Also in the interest of time (and GPU resources), the library focuses on lightweight gradient-based NAS algorithms, which allows us to run the complete optimization with a relatively small number of GPU hours.
- As we will see later, PLiNIO can export ONNX files in the format expected by our DNN compiler (MATCH) and target hardware (GAP9).
- We developed the library internally, so we should be able to solve issues if they occur (hopefully....)

As you can read in the documentation, PLiNIO supports multiple types of optimization, with a common API. In general, optimizing a model with PLiNIO requires three basic steps.

- Converting a PyTorch model (an `nn.Module` instance), optionally equipped with special layers, to an optimizable format
- Training the converted model, with a modified loss function that considers both the model's accuracy and its cost (e.g. in terms of number of parameters).
- Exporting the final output of the optimization again as a standard pytorch `nn.Module`

In this session, we will use one specific method from PLiNIO called **PIT** (Pruning-in-Time). Other supported optimizations include SuperNet-based Differentiable NAS and Quantization/Mixed-Precision Search, which we will use in a later session.



## The PIT Algorithm

PIT is essentially a channel-based pruning algorithm, and is described in details [here](https://arxiv.org/abs/2301.10281). During training, it learns to mask out unimportant channels for each layer, effectively eliminating them from the network. The working principle is schematized in the following picture:

<br>
<center><img src="./assets/pit.png" alt="PIT Algorithm" class="bg-primary" width="50%"></center>
<center> Fig. 4: The PIT Algorithm</center>
<br>

The "seed" DNN (a standard network) is turned into a "searchable" DNN by associating each supported layer (convolutional or linear) with a new set of trainable *binary* parameters $\theta$. Each $\theta_i$ is associated with a single weight filter/neuron, and functions as a mask. When it is set to $\theta_i=1$, the filter/neuron is kept, whereas, when it is set to $\theta_i =0 $ it is removed from the layer's output. Thus, the corresponding output activation channel is effectively eliminated. Notably, PIT can also optimize other parameters of 1D Convolutions (namely the kernel size and the dilation). But since we turned our network into a 2D one for compatibility with later steps, we won't use those features here. You're of course free to try them on other problems in the future, as well as the other optimizations in PLiNIO (and contact us for support).

The $\theta$ values are trained with gradient-descent together with the normal weights of the network. Similarly to a SuperNet DNAS, the goal is to optimize a combined loss function in the form:

$$
\mathcal{L}_{tot}(W,\theta) = \mathcal{L}_{task}(W,\theta) + \lambda \mathcal{L}_{cost}(\theta)
$$

where $\mathcal{L}_{task}$ is the standard task-loss (cross-entropy in our case), while $\mathcal{L}_{cost}$ encodes the cost of the network. In fact, although PIT essentially implements channel pruning, it can also be considered a form of "mask-based DNAS". In the following, we will use model size (n. of parameters) as cost metric. This optimization goal ensures that PIT only eliminates unimportant channels. Moreover, applying it during training allows to adapt the weights $W$ to recover from the accuracy drop caused by the eliminated DNN portions.

Let's import the PLiNIO classes needed to implement this optimization:

In [20]:
from plinio.methods import PIT
from plinio.cost import params

## Preparing for the Optimization

To use PIT, the first step consists in converting our model into an optimizable form. This is done with the class constructor `PIT()`, and essentially creates the "searchable" network from the figure above. The constructor takes three main parameters:
- A DNN model in standard `nn.Module` form (better if pre-trained on the target task). We will use the just-trained TEMPONet here.
- The shape of a single input sample (needed for internal graph analysis passes).
- A cost estimator, that is, the metric that we want to consider as "DNN cost". In our case, it will be the number of parameters (`params`).

There are other optional parameters, that you can ignore for now. Just make sure that you set `discrete_cost=True` to have a more accurate cost estimate.

Let's define a function to get a fresh PIT model. As before, we make sure to always start from the same identical model, by re-seeding our code just before the constructor.

<div class="alert alert-block alert-warning">
<b>Task:</b> Call the PIT() constructor with the parameters described above. Then move the newly created model to the training device (CPU or GPU).
</div>

In [21]:
def get_pit_model(original_model):
    model_copy = copy.deepcopy(original_model)
    # call the PIT() constructor on model_copy (this ensures that the original model is not touched).
    # Then move the pit model to the training device. Lastly, return the converted model
    # (expected: 3 lines)
    # YOUR_CODE_START



    # YOUR_CODE_END

You can now call the function just defined, and get an estimate of the cost of the network by simply looking at the `model.cost` attribute of the PIT model. Let's try it:
<div class="alert alert-block alert-warning">
<b>Task:</b> Get and print the DNN cost estimate according to PLiNIO.
</div>


In [22]:
pit_model = get_pit_model(model)
# print the initial model cost (expected: 1 line)
# YOUR_CODE_START

# YOUR_CODE_END

Notice that the cost estimate is slightly smaller (by a few thousands of parameters) compared to the one provided by `torchinfo`. This is because the PIT estimate only takes into account layers that PLiNIO is able to optimize (Conv, Linear, etc). Other layers (typically negligile) are ignored. Importantly, calling `.cost` on a PLiNIO model internally invokes a **differentiable** PyTorch function. Therefore, we can apply back-propagation to it. So, this will become our $\mathcal{L}_{cost}$ term in the loss function. 

Another key parameter to set in the loss is the regularization strength $\lambda$. Choosing a value for this scalar is difficult. However, a good rule of thumb is to consider values so that:

$$
\lambda \cdot \mathcal{L_{cost}} \simeq \mathcal{L_{task}}
$$

Since our initial cost estimate is around 500k, and the final validation loss after warmup was roughly 1.7, values around $10^{-6}$ are a good choice. Let us define an array of $\lambda$ values to try. Considering the long training times, we may evaluate 2 or 3 values at most in the time of this hands-on:

<div class="alert alert-block alert-warning">
<b>Task:</b> Create a list with regularization strength values that you'd like to try.
</div>


In [23]:
# create a list called REG_STRENGTHS, with the desired regularization strengths lambda values
# to consider in the optimization (expected: 1 line)
# YOUR_CODE_START

# YOUR_CODE_END

We will use two different optimizers for the DNN weights $W$ and for the $\theta$ masks.
Part of the reason for this is that we should not apply *weight decay* to the masks, as this would lead to pruning all channels, even the important ones. Moreover, we will also use two different learning rates, so that the masks are updated faster (also helps speeding-up this phase a bit). 

PLiNIO offers a convenient API to get only the $W$, with the `model.net_parameters()` method, or only the $\theta$, with `model.nas_parameters()`. Let's use these two methods to define a function that returns two Adam optimizer instances, one for the weights and one for the architectural parameters. For the latter, set the `weight_decay` optional parameter to 0 and use the `config['nas_learning_rate']` value as its learning rate. Note that, in general, we could have also used two separate optimizers (e.g. Adam and SGD) if needed. There's not a golden rule here, it often boils down to trial and error (or using a HPO method).

<div class="alert alert-block alert-warning">
<b>Task:</b> Complete the function below to return two optimizers, one for the weights and one for the masks
</div>



In [24]:
def get_nas_optimizers(model, config):
    # create and return two optimizers, one standard Adam instance for the net_parameters() and another
    # Adam instance for the nas_parameters(), with two different learning rates (specified in config)
    # The second optimizer should have weight_decay set to 0 (expected: 2 lines)
    # YOUR_CODE_START


    # YOUR_CODE_END
    return net_optimizer, arch_optimizer

Lastly, let's define a modified training loop for the optimization. This is similar to a standard training loop, with some key differences:

- The function shall use the two separate optimizers for $W$ and $\theta$ just created.
- We have to define a local function `nas_criterion` that combines the two loss terms $\mathcal{L_{task}}$ and $\mathcal{L_{cost}}$ using the regularization strength $\lambda$.
- In each epoch, we first train the $W$ on the training set, then train the $\theta$ on the validation set, finally evaluate (again on the validation set) and repeat.


To make sure that the iteration only training set only affects the $W$, and the one on the validation set only the $\theta$, you can use two utility functions in PLiNIO, respectively `train_net_only()` and `train_nas_only()`. These ensure that gradients are only propagated to DNN weights and masks respectively. There is also a third method `train_net_and_nas()` to enable gradient propagation to both sets of parameters simultaneously.

<div class="alert alert-block alert-warning">
<b>Task:</b> Complete the function below to implement the optimization loop, following the instructions of this cell as well as the comments in the code.
</div>

In [25]:
def nas_loop(checkpoint_dir, config, model, criterion, train_dl, val_dl, device, num_classes=9, file=sys.stdout):

    # same as before: skip the training if best checkpoint exists.
    if try_load_checkpoint(model, checkpoint_dir, device):
        print(f"Skipping NAS and loading pre-cooked model from: {checkpoint_dir}")
        return None

    # initialize the two optimizers, with the newly defined function (expected: 1 line) 
    # YOUR_CODE_START

    # YOUR_CODE_END
    checkpoint = CheckPoint(checkpoint_dir, model, net_optimizer, 'max', fmt='ck_{epoch:03d}.ckp')
    earlystop = EarlyStopping(patience=config['patience'], mode='max')
    history = []

    # complete this function to return L_task(output, target) + lambda * L_cost
    # remember that L_task is called "criterion", lambda is part of the config dictionary
    # and L_cost is obtained from the PLiNIO class as model.cost (expected: 1 line)
    def nas_criterion(output, target):
        # YOUR_CODE_START

        # YOUR_CODE_END
    
    for epoch in range(config['epochs']):
        # set the PLiNIO model to train only the normal weights, and then train it for one epoch
        # on the training set, reusing the function from the original training loop (expected: 2 lines)
        # YOUR_CODE_START


        # YOUR_CODE_END
        
        # set the PLiNIO model to train only the architectural parameters (masks), and then train it for one epoch
        # on the VALIDATION set, reusing the function from the original training loop.
        # Careful: here we need to use a different optimizer, and a different criterion too 
        # i.e., the combined loss defined above (expected: 2 lines)
        # YOUR_CODE_START


        # YOUR_CODE_END
        
        # evaluate the model on the validation set, reusing the function from the original training loop
        # (expected: 1 line)
        # YOUR_CODE_START

        # YOUR_CODE_END

        cost = model.cost.detach().cpu().numpy()
        # log all metrics, as well as the model cost, to the console or to a file.
        # also append these values to the history list as a tuple
        history.append((epoch+1, n_loss, t_loss, t_acc, t_macro_acc, v_loss, v_acc, v_macro_acc, cost))
        print(f"Epoch: {epoch+1}, Total NAS Loss: {n_loss:.2f}, ",
              f"Task Loss: {t_loss:.2f}, Acc: {t_acc:.2f}, Macro Acc: {t_macro_acc:.2f}",
              f"Val Task Loss: {v_loss:.2f}, Val Acc: {v_acc:.2f}, Val Macro Acc: {v_macro_acc:.2f}, ",
              f"Model Cost: {cost}", file=file)

        if epoch > 100:
            # save checkpoint based on the validation macro accuracy, but only after the
            # first 100 epochs, leaving time to the NAS to modify the architecture
            checkpoint(epoch, v_macro_acc)
            
            # check if we need to early-stop, with the same rationale as checkpointing
            if earlystop(v_macro_acc):
                print("Stopped at epoch {} because of early stopping".format(epoch + 1))
                break

    # create final history dataframe
    cols = ['epoch','nas_loss', 'loss','acc','macro_acc', 'val_loss', 'val_acc', 'val_macro_acc', 'cost']
    history = pd.DataFrame(history, columns=cols)

    # restore the best model and save it as "best.ckp" for later
    checkpoint.load_best()
    checkpoint.save(checkpoint_dir / 'best.ckp')

    return history

## Running the Optimization

Let's use the previously defined functions to run our first NAS optimization. In particular, let's build a loop that calls the training function repeatedly, changing the regularization strength parameter $\lambda$.
For each training, let's collect the final accuracy on the test set, and the corresponding model size in two arrays. We will later use these arrays to plot a Pareto front.

Importantly, model selection should *always* be performed on the *validation* set, not on the *test* set, which should only be used for final evaluation of the optimized model. We will follow this best practice here.

<div class="alert alert-block alert-warning">
<b>Task:</b> Complete the function below to iteratively run the optimization loop with different regularization strength values, and build two lists containing the macro average accuracy on the validation set and number of parameters, for the seed and for all models found by PIT. Follow the comments in the code.
</div>

In [ ]:
macro_acc_list = []
params_list = []

# add the seed model's macro_acc (on the VALIDATION set) and its number of parameters as the first
# elements of the two lists 
_, _, seed_macro_acc = evaluate(pit_model, criterion, val_dl, device, num_classes=9)
seed_size = pit_model.cost.detach().cpu().numpy()
macro_acc_list.append(seed_macro_acc)
params_list.append(seed_size)

for strength in REG_STRENGTHS:
    print(f"Training with strength {strength}")

    # generate a fresh PIT model in each iteration calling the appropriate function
    # defined above
    # (expected: 1 lines)
    # YOUR_CODE_START

    # YOUR_CODE_END

    # generate a fresh instance of the dataloaders
    train_dl, val_dl, test_dl = build_dataloaders(TRAINING_CONFIG, train_ds, val_ds, test_ds, num_workers=4)
    
    # generate a fresh instance of the loss function
    criterion = get_criterion(train_dl, TRAINING_CONFIG, device)

    # update the training config setting the correct value for lambda
    TRAINING_CONFIG.update({'reg_strength': strength})

    # run the NAS loop defined above, saving the checkpoints under "SAVE_DIR/pit_nas_<STRENGTH_VALUE>".
    # you can suppress logging if you want, by setting the file parameter to /dev/null (expected: 1 line)
    # YOUR_CODE_START


    # YOUR_CODE_END

    # perform a last evaluation of the optimized model on the VALIDATION set, print it
    # to the console, and append the macro accuracy to macro_acc_list. Also, append the final
    # value of model.cost to the params_list (expected: 4 lines)
    # YOUR_CODE_START




    # YOUR_CODE_END
    

We can visualize the output of our design-space exploration in the space of possible DNN architectures by plotting the results in an accuracy versus model size plane. The next function does exactly that. It takes three arrays containing the list of model sizes, macro average accuracies and regularization strengths for the tested models. It also assumes that the initial elements of the first two arrays refer to the Seed model. Let's call it and visualize the results: 

In [ ]:
plot_pareto(params_list, macro_acc_list, REG_STRENGTHS)

## Final Model Selection and Fine-Tuning

Looking at the graph above, we now want to select one architecture that seems promising and fine-tune it further on our dataset, to see if we can match the accuracy of the seed.


<div class="alert alert-block alert-warning">
<b>Task:</b> Select a strength value and re-load the best model found during the PIT Optimization.
</div>


In [ ]:
# define a variable called SELECTED_STRENGTH with the strength value you want to
# use for fine-tuning (expected: 1 line)
# YOUR_CODE_START

# YOUR_CODE_END

# load the selected model and evaluate it again on the validation set
ret = try_load_checkpoint(pit_model, SAVE_DIR / f"pit_nas_{SELECTED_STRENGTH}", device)
_, _, val_macro_acc = evaluate(pit_model, criterion, val_dl, device, num_classes=9)
print(f"Val Macro Acc after re-loading: {val_macro_acc}")

This model is still an instance of the PIT class. In order to obtain a standard `nn.Module` that we can use with any other torch-based tool (such as `torchinfo`), we can use  the `model.export()` API of PLiNIO. Please ensure that both net weights $W$ and NAS weights $\theta$ are set to trainable before exporting.


<div class="alert alert-block alert-warning">
<b>Task:</b> Export the final optimized model.
</div>

In [ ]:
# set net and nas parameters to trainable, export the model and move it to the training device
# (expected: 3 lines).
# YOUR_CODE_START



# YOUR_CODE_END

Let's look at the architecture of the optimized model using `torchinfo`.

In [ ]:
# print the exported model summary
print(summary(final_model, (1,) + input_shape))

<div class="alert alert-block alert-info">
<b>Question:</b> Look at the optimized architecture. What changed? Which layers have been affected the most?
</div>

Using the exported model as is will incur some accuracy drop. This is because of some graph conversion passes happening during export (e.g. Batch Normalization unfolding). However, fine-tuning our model for some epochs should allow us to recover, and often even improve, the accuracy obtained at the end of the search phase. Let's do that!!

<div class="alert alert-block alert-warning">
<b>Task:</b> Use the next cell to train the final_model for some epochs. Remember that this is a normal PyTorch DNN, so you can reuse 100% of the code written initially for the seed.
</div>

In [ ]:
# generate fresh "dataloaders" and "criterion", and use the training_loop() function to train the 
# optimized model for a while save the checkpoints under "SAVE_DIR/pit_finetune_<STRENGTH_VALUE>"
# (expected: 3 lines)
# YOUR_CODE_START



# YOUR_CODE_END

Finally, let's evaluate our optimized model on the test set, and visualize the confusion matrix.

In [ ]:
# evaluate the final model on the test set, printing the results, and also visualize the confusion matrix
test_loss, test_acc, test_macro_acc = evaluate(final_model, criterion, test_dl, device, num_classes=9)
print(f"Final Optimized Model Test Loss: {test_loss:.2f}, Test Acc: {test_acc:.2f}, Test Macro Acc: {test_macro_acc:.2f}")
plot_conf_matrix(model, test_dl, device)

<div class="alert alert-block alert-info">
<b>Question:</b> Reflect and summarize what we have done in this session. If you did everything correctly, you should have obtained a DNN that approximately matches the test accuracy of the original one with a fraction of the parameters. How smaller is your optimized model exactly? Would this model fit in the on-chip memory of GAP9?
</div>

## Saving the Final Model

Let's save the model in a separate location to reuse it more easily in later sessions:

In [ ]:
torch.save(final_model, SAVE_DIR / f'final_model_{SELECTED_STRENGTH}.pt')

<div class="alert alert-block alert-danger">
<b>IMPORTANT:</b> for sake of time, each group will generate a single Optimized DNN (rather than a full Pareto front) during the hands-on session. Read below for details!
</div>

The instructors will tell you what value to try. Once done, you will have to upload the optimized **and finetuned** model to [this](https://www.dropbox.com/request/IRUUGGAlAZ4ShAWPr8MF) link, specifying your group name (pick the name that you want, but no vulgarity please &#128515;). 

We will then pick one lucky winner DNN, which *all groups* will use for the personalized fine-tuning in Hands-on #3. Once selected, we will put the winner in [this](https://www.dropbox.com/scl/fo/17nahcdckiig6b5wagk7d/ANIurZ2izCXPrXjapr_l4s8?rlkey=fc9erl3lyq509puspy35ikc3h&st=ge5f8ywd&dl=0) folder for you to download.
</div>

# Extras

If you managed to reach the end of this notebook before the end of the session, here are some extra things you could try:
- Modify the training parameters for the Seed (and for the PIT optimization) to reduce over-fitting.
- Try different regularization strength values to enrich your Pareto front of models (Note that for this particular task, there's a quite "steep" decline in accuracy for strengths higher than 2e-5 (at least with our current training parameters).
- Try to apply the same optimization steps on data from a single patient. What do you expect would change?
- Feel free to explore PLiNIO and its other optimizations (e.g. SuperNet). Admittedly, the documentation could be improved, but don't hesitate to ask us if you have doubts..
- What about a different seed architecture?

